# <center>⚡️🏁 [Welcome!](https://stepik.org/lesson/1062722/step/2?unit=1072411) Как запустить эти ваши ноутбуки с курса? 🤔📔 </center>

<img src='../images/choice_api.jpeg' align="right" width="400" height="400" >

## Оглавление ноутбука

 * [🚀 API от OpenAI.](#part1)
 * [🛠 API от OpenAI с __нашего сервера__.](#part2)
 * [🤐 Чат-модели и объект ChatOpenAI](#part3)
 * [🔧 Open Source LLM модели и API от HuggingFaceHub.](#part4)
 * [🔋 Локальный запуск LLM без интернета и API ключей.](#part5)
 * [🧸 Выводы и заключения.](#part6)

In [2]:
import os
import warnings
from getpass import getpass  # Для скрытого ввода API ключей

warnings.filterwarnings("ignore")

#### Для любого варианта использования сначала обязательно установить необходимые библиотеки!

In [ ]:
# Установим нужные библиотеки (langchain>=0.3.25)
!pip install openai langchain huggingface_hub langchain-openai langchain-community transformers langchain-huggingface -U -q

# <center id="part1">  🤖 Как использовать `API` от `OpenAI` напрямую?!

<div style="background-color:#fff0ff; padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
Если хотите использовать официальное <code>API</code> от <code>OpenAI</code> в РФ, то используйте <code>VPN</code> или запускайте код через <code>Google Colab</code>.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
**📖  Получение ключа для использования `OpenAI API` 🔑**
> При создании своего первого ключа, вы получите бесплатные 5$ для использования платформы.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
0. Если вы из РФ, то используйте __VPN__ при заходе на сайт. 
1. Для регистрации аккаунта OpenAI потребуется __зарубежная симкарта__ (можно приобрести в интернете, либо купить доступ к уже зарегистрированному аккаунту, ~100 руб.
2. Перейдите по [ссылке](https://platform.openai.com/account/api-keys) и авторизуйтесь на сайте `OpenAI`.
3. После успешной авторизации на сайте, нажмите на кнопку `+ Create new secret key` и введите любое название для ключа.
4. Скопируйте полученный ключ и поместите его в переменную среды `OPENAI_API_KEY`
5. Для оплаты новых токенов из РФ, потребуется __иностранная карта__!

* ⚙️ Начнем с прямых вызовов к API от OpenAI.
* 🗂 [Документация](https://platform.openai.com/docs/api-reference/making-requests) по использованию API (открывается с VPN)
* 👨‍🍳 Полезный [Cookbook](https://cookbook.openai.com/)
* 🎁 Ниже мы покажем способ, как использовать курс от авторов курса

## Куда вводить ключи: <br>

<div style="background-color:#fff0ff; padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
**1 вариант** - В появившемся окне после выполнения ячейки с `getpass` - более безопасно, но придется вводить ключ при каждом перезапуске ноутбука.

<img src='../images/wind.png' width="600" height="300" >

In [5]:
# Ключ следует вводить в появившемся окне после запуска ячейки и нажать Enter
# Либо раскомментировать строку ниже и записать ключ в переменную (менее безопасно)

# openai_api_key = "Введите ваш OpenAI API-ключ"
openai_api_key = getpass(prompt="Введите ваш OpenAI API-ключ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Введите ваш OpenAI API-ключ ········


<div style="background-color:#fff0ff; padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

**2 вариант** - Если ноутбуком будете пользоваться только вы, и точно уверены, что ноутбук с ключом не будет доступен третьим лицам. Можно постоянно не вводить ключ.

<img src='../images/wind2.png' width="600" height="300" >

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Рассмотрим некоторые параметры функции `.chat.completions.create()`

О всех параметрах можно узнать на [странице](https://platform.openai.com/docs/api-reference/chat/create)

In [7]:
import openai
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
)

#print(client.models.list()) # Выведет все возможные модели от OpenAI, которые можно указать в параметре model

In [8]:
messages = [
    {
        "role": "user",  # Роль - ассистент или юзер
        "content": "1+1",  # Сам промпт для подачи в chatGPT
    }
]

chat_completion = client.chat.completions.create(
    max_tokens=100,
    model="gpt-3.5-turbo",  # модель для выбора
    messages=messages,  # сообщение
    temperature=0,  # степень креативности ответа
)

print(chat_completion.choices[0].message.content)

2


<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

**😎 Два обязательных параметра**
    
- `messages` - список объектов сообщений, где каждый объект имеет два обязательных поля:
    - `content` - содержимое вашего сообщения
    - `role` - роль автора сообщений (`system`, `user` или `assistant`)
- `model` - модель, используемая для генерации сообщения (доступные модели на [странице](https://platform.openai.com/docs/models/model-endpoint-compatibility))

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

**🥸 Полезно использовать**
    
- `max_tokens` - максимальное количество токенов, генерируемых в сообщении
- `n` - сколько вариантов ответа должно быть сгенерировано для каждого входного сообщения (по умолчанию 1)
- `temperature` - влияет на степень случайности генерации (принимает значения от 0 до 2, по умолчанию 1). Высокое значение температуры делает вероятности всех слов в выводе более равномерными, что приводит к более случайному и разнообразному тексту. При низком значении температуры модель будет склонна генерировать более предсказуемый и детерминированный текст.
- `top_p` - тоже влияет на степень случайности генерации. Определяет, какой процент наиболее вероятных слов будет включен в генерацию. Например, если top-p равно 0.8, то модель будет использовать вероятности слов так, чтобы включить в генерацию слова, составляющие 80% верхней части распределения вероятностей.

> Рекомендуем изменять либо параметр `temperature`, либо `top_p`, но не оба одновременно

```python
{
  "id": "chatcmpl-8B6YmCk5OiK7Z6RkRrkx72mEQaPGF",
  "object": "chat.completion",
  "created": 1697657516,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "1+1 equals 2."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 10,
    "completion_tokens": 7,
    "total_tokens": 17
  }
}
```

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

**🧐 А что получается в выводе?**

- `id` - уникальный идентификатор запроса
- `object` - тип возвращаемого объекта
- `created` - время запроса в timestamp-формате
- `model` - полное название модели, которая использовалась для генерации ответа
- `choices` - список объектов ответа (только один, если не изменяли параметр `n`)
    - `index` - индекс ответа в списке вариантов
    - `message` - объект сообщения, сгенерированный моделью, содержит `role` и `content`
    - `finish_reason` - причина, по которой модель закончила генерировать ответ
        - `stop` - достигнута естественная точка остановки
        - `length` - достигнуто максимальное количество токенов, указанное в запросе
        - `content_filter` - сгенерированный текст содержит нежелательный материал, который нарушает правила
- `usage` - количество используемых токенов
    - `prompt_tokens` - для распознавания вопроса
    - `completition_tokens` - для генерации ответа
    - `total_tokens` - для всего запроса (вопрос + ответ)

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
✅ __Все!__ Этого уже достаточно, чтобы просто получать ответы от `ChatGPT` кодом. Если для ваших задач этого хватает, то можно использовать уже так. Для удобства, можно еще обернуть дополнительно в функцию.

# <center id="part2"> 🤖 `API` от `OpenAI` с ключом курса (рекомендуется)

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

**📖 Получение ключа для использования модели на нашем сервере** 🎁🔑

1. Перейдите по [ссылке](https://t.me/llm_course_bot) в нашего телеграмм-бота.
2. Авторизуйтесь, указав в нем свой `Stepik id`, с которым занимаетесь на курсе.
3. Получите свой ключ и используйте наш API для прохождения курса.
4. Также в боте можно проверить действие вашего ключа, узнать баланс и дату, когда ключ перестанет действовать.

> Если возникли трудности, ошибки, вопросы или бот не работает, то смело пишите в чат курса, мы вам поможем.

In [7]:
# Ключ следует вводить в появившемся окне после запуска ячейки
# Либо раскомментировать строку ниже и записать ключ в переменную (менее безопасно)

# course_api_key = 'Введите ваш API-ключ, который получили в боте @llm_course_bot'
course_api_key = getpass(prompt="Введите ваш API-ключ, который получили в тг боте @llm_course_bot")

Введите ваш API-ключ, который получили в тг боте @llm_course_bot ········


In [ ]:
from openai import OpenAI

# Добавляем в клиент ключ курса и url-адрес нашего сервера
client = OpenAI(api_key=course_api_key, base_url="https://aleron-llm.neuraldeep.tech")

In [13]:
prompt = "1+1="

messages = [
    {
        "role": "user",  # Роль - ассистент или юзер
        "content": prompt ,  # Сам промпт для подачи в ChatGPT
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",  # модель для выбора
    messages=messages,  # сообщение
    temperature=0,  # степень креативности ответа
)

print(response.choices[0].message.content)

1 + 1 = 2.


<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
✅ Работает на 100% также!


# <center id="part3"> 🤐 Чат модели и объект `ChatOpenAI` от `LangChain`
#### <center> (Ячейка ниже работает только с официальным ключом от OpenAI)

<div style="background-color:#fff0ff; padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
📖 Чтобы спользовать официальное <code>API</code> от <code>OpenAI</code> в РФ, то используйте <code>VPN</code> или запускайте код через <code>Google Colab</code>.

In [14]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

template = """Вопрос: {question}
Ответ: Дай короткий ответ"""

prompt = PromptTemplate(template=template, input_variables=["question"])

openai_llm = ChatOpenAI()

llm_chain = prompt | openai_llm

question = "Когда человек первый раз полетел в космос?"

print(llm_chain.invoke(question).content)

12 апреля 1961 года.


# <center> 🤘 Чат модель `ChatOpenAI` с ключом курса (рекомендуется)

In [14]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

template = """Вопрос: {question}
Ответ: Дай короткий ответ
"""

prompt = PromptTemplate(template=template, input_variables=["question"])

# Добавляем ключ курса и url-адрес нашего сервера
openai_llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini', 
                        base_url="https://aleron-llm.neuraldeep.tech/")

llm_chain = prompt | openai_llm

question = "Когда человек первый раз полетел в космос?"

print(llm_chain.invoke(question).content)

Первый раз человек полетел в космос 12 апреля 1961 года. Это был Юрий Гагарин на корабле «Восток-1».


# <center id="part4"> 🤗 Исследуем мир Open Source: `API` от `HuggingFaceHub`

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
**📖 Получение ключа для использования `HuggingFaceHub` `API`**

1. Авторизуйтесь или зарегистрируйтесь на сайте [HuggingFace](https://huggingface.co/).
2. Перейдите по [ссылке](https://huggingface.co/settings/tokens) и нажмите на кнопку `New token`.
3. Введите любое название и нажмите на кнопку `Generate a token`.
4. Скопируйте полученный ключ и поместите его в переменную среды `HUGGINGFACEHUB_API_TOKEN`


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
**Важно:** Не все модели имеют бесплатный API-инференс на `HuggingFace`. Как правило, это небольшие модели. Также бесплатный инференс имеет ограниченное контекстное окно (примерно 200 токенов). Проверить имеет ли модель бесплатный инференс по API можно в ее карточке.

In [3]:
# Ключ следует вводить в появившемся окне после запуска ячейки
# Либо раскомментировать строку ниже и записать ключ в переменную (менее безопасно)

# os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'Введите ваш HuggingFaceHub API-ключ'
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass(prompt="Введите ваш HuggingFaceHub API-ключ")

Введите ваш HuggingFaceHub API-ключ ········


# <center> ВРЕМЕННО НЕ РАБОТАЕТ (обновляется HugginFace)

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate

# initialize HF LLM
hf_llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen3-32B", # вводим название модели с HuggingFace
    #repo_id="mistralai/Mistral-Nemo-Base-2407",
    #provider="novita",
    #task='conversational',
    max_new_tokens=50,
)

# build prompt template for simple question-answering
template = """Question: {question}.

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = prompt | hf_llm

question = "When did man first fly into space?"

print(llm_chain.invoke(question))

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
✅ __Все!__ теперь мы можете даже не спускать свои деньги на токены! Выбирайте любую хайповую модельку и кайфуйте! Нужен только токен.<br>
**!!!** В бесплатном HF API контекстное окно для запроса к модели ограничено 250 токенами.

<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

Если код не запустился, проверьте [актуальный список](https://huggingface.co/models?pipeline_tag=text-generation&inference_provider=hf-inference&sort=trending) моделей с бесплатным инференсом от`HuggingFace` и замените название модели на одно из доступных.

# <center id="part5"> 💻 Ваш собственный ChatGPT: запуск LLM локально

> Для локального использования моделей `HuggingFace` (на вашем локальном компьютере или сервере), необходима библиотека `transformers`, а также должны быть скачаны веса модели (скачиваются при первом запуске, далее загружаются с локального хранилища).

<div class="alert alert-success" style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

**Важно:** Веса не всех моделей сразу доступны для скачивания, некоторые из них защищены лицензиями, и чтобы начать ими пользоваться необходимо сначала принять условие лицензии на `HuggingFace`. Например, популярное семейство моделей `LAMA` требует предварительного принятия лицензионного соглашения. 

<div class="alert alert-success" style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
    
Более подробно про Open Source модели и как их выбирать расскажем в 5 модуле курса!

In [20]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate

bloom = HuggingFacePipeline.from_model_id(
    model_id="bigscience/bloom-1b7",  # Название и версия модели, позже покажем вам весь LLM-зоопарк
    task="text-generation",  # Вид задачи, в нашем случае - генерация текста
    model_kwargs={
        "temperature": 1,
        "max_length": 64 },  # все те же креативность и максимальная длина
    #device=0,  # Номер GPU карточки, если есть! Раскомментируйте, если имеете доступ к GPU!
)

Device set to use cuda:0


In [21]:
template = """Question: {question}.

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = prompt | bloom

question = "When did man first fly into space?"

print(llm_chain.invoke(question))

Question: When did man first fly into space?.

Answer: The first man to fly into space was the American astronaut
John Glenn, who was born in New York City on January 1, 1903. He was
the son of a carpenter and a housewife. He was a very intelligent boy,
and


<div class="alert alert-info" style="padding:10px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

✅ __Все!__ Теперь вам даже интерент не нужен. Грузите себе локально и пользуйтесь! Однако, следите за качеством выбираемой модели. Как убедились выше, не у всех моделей качество генерации высокое - маленькая модель `bloom` на 1 миллиард параметров, не знает про Гагарина!

# <center id="part6"> 🧸 Выводы и заключения ✅
<div style="background-color:#e6ffe6; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

* ✅ Вы умеете передавать запросы к LLM через API
* ✅ Вы знаете откуда стягивать модели 🤗
* 🔑 Даже если у вас нет VPN, иностранного номера и счета, то у вас есть ключ от команды курса!
* ✅ Под каждую задачу есть своя оптимальная модель подходящего качества и размера, не одним ChatGPT едины!
* ✅ У API разных LLM много неочевидных трюков, про которые будем говорить дальше в курсе